# NYC Taxi Trip Duration — Phase 1 (Neural Networks)

This notebook satisfies **Phase 1: Neural Networks (NN)** deliverables:

- **RQ1:** How well can a neural network predict trip duration from tabular features?
- **RQ2:** Which features contribute most to predictive performance?
- **H1:** A properly tuned neural network will outperform a classical baseline (e.g., Ridge regression).

**Important evaluation note (heavy‑tailed target):**
Trip duration is heavy‑tailed. We train on `log1p(trip_duration)` and report:
- **Log-space metrics** (recommended for model selection): **R²_log**, **RMSE_log**
- **Original-scale metrics** (interpretability): **RMSE**, **MAE**, **MAPE**, **R²**
- For the “normalized” comparison requested by the rubric: **R²_log + MAPE** on holdout.


In [1]:
from pathlib import Path
import urllib.request
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

SEED = 42
NROWS = 1_000_000
TARGET = "trip_duration"

# Kaggle NYC Taxi Trip Duration schema (train.csv)
DATA_URL = "https://github.com/DrAlzahraniProjects/csusb_spring26_cse5140_team1/releases/download/v1.0/train.csv"
DATA_PATH = Path("../data/train.csv")

ARTIFACTS_DIR = Path("../artifacts")
ARTIFACTS_DIR.mkdir(parents=True, exist_ok=True)

print("Config:")
print("  DATA_PATH:", DATA_PATH)
print("  NROWS:", NROWS)
print("  SEED:", SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Config:
  DATA_PATH: ../data/train.csv
  NROWS: 1000000
  SEED: 42
Device: cpu


In [2]:
import urllib.request

# Make sure the directory exists
DATA_PATH.parent.mkdir(parents=True, exist_ok=True)

if not DATA_PATH.exists():
    print("Downloading dataset...")
    urllib.request.urlretrieve(DATA_URL, DATA_PATH)
    print("Download complete.")
else:
    print("Dataset already exists. Skipping download.")

print("Loading dataset into memory...")
df = pd.read_csv(DATA_PATH, nrows=NROWS)
print("Loaded df:", df.shape)
df.head()



Dataset already exists. Skipping download.
Loading dataset into memory...
Loaded df: (1000000, 11)


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [3]:
def seed_everything(seed=SEED):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)


In [4]:
# Load 1,000,000 rows and shuffle deterministically
df = pd.read_csv(DATA_PATH, nrows=NROWS)
print("Loaded:", df.shape)

df = df.sample(frac=1.0, random_state=SEED).reset_index(drop=True)
print("Shuffled with seed:", SEED)

df.head()


Loaded: (1000000, 11)
Shuffled with seed: 42


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id3435429,1,2016-02-18 22:34:53,2016-02-18 22:49:37,1,-73.956161,40.694500,-73.987869,40.720985,N,884
1,id2267606,1,2016-05-14 14:37:43,2016-05-14 14:52:09,1,-73.975922,40.757133,-73.950813,40.770882,N,866
2,id3771460,1,2016-06-15 01:42:25,2016-06-15 01:52:02,1,-73.982391,40.762222,-73.952019,40.777706,N,577
3,id2766058,2,2016-03-21 22:37:26,2016-03-21 22:42:10,2,-73.998482,40.740463,-74.004646,40.722782,N,284
4,id2834780,2,2016-06-07 21:33:57,2016-06-07 21:36:06,1,-73.998169,40.735550,-73.991913,40.744041,N,129


In [5]:
# Splits:
# - 50% final holdout (never used until final evaluation)
# - remaining 50% dev split into train/val (2/3 train, 1/3 val)

dev_df, holdout_df = train_test_split(df, test_size=0.50, random_state=SEED)
train_df, val_df = train_test_split(dev_df, test_size=1/3, random_state=SEED)

print("train_df:", train_df.shape)
print("val_df:", val_df.shape)
print("holdout_df:", holdout_df.shape)


train_df: (333333, 11)
val_df: (166667, 11)
holdout_df: (500000, 11)


## Feature families (temporal + spatial/distance + proxies)

In [6]:
def haversine_km(lat1, lon1, lat2, lon2):
    """Vectorized Haversine distance in km."""
    R = 6371.0
    lat1 = np.radians(lat1); lon1 = np.radians(lon1)
    lat2 = np.radians(lat2); lon2 = np.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    return 2 * R * np.arcsin(np.sqrt(a))

def build_features(dfin: pd.DataFrame) -> pd.DataFrame:
    X = pd.DataFrame(index=dfin.index)

    # Temporal
    dt = pd.to_datetime(dfin["pickup_datetime"], errors="coerce")
    X["pickup_hour"]  = dt.dt.hour.fillna(0).astype(int)
    X["pickup_dow"]   = dt.dt.dayofweek.fillna(0).astype(int)
    X["pickup_month"] = dt.dt.month.fillna(0).astype(int)

    X["hour_sin"] = np.sin(2*np.pi*X["pickup_hour"]/24)
    X["hour_cos"] = np.cos(2*np.pi*X["pickup_hour"]/24)
    X["dow_sin"]  = np.sin(2*np.pi*X["pickup_dow"]/7)
    X["dow_cos"]  = np.cos(2*np.pi*X["pickup_dow"]/7)

    # Spatial
    X["delta_lat"] = (dfin["dropoff_latitude"] - dfin["pickup_latitude"]).astype(float)
    X["delta_lon"] = (dfin["dropoff_longitude"] - dfin["pickup_longitude"]).astype(float)
    X["haversine_km"] = haversine_km(
        dfin["pickup_latitude"].astype(float),
        dfin["pickup_longitude"].astype(float),
        dfin["dropoff_latitude"].astype(float),
        dfin["dropoff_longitude"].astype(float),
    )

    # Proxies
    X["passenger_count"] = pd.to_numeric(dfin["passenger_count"], errors="coerce").fillna(0.0)
    X["store_and_fwd_Y"] = (dfin["store_and_fwd_flag"].astype(str).str.upper() == "Y").astype(int)

    vendor_oh = pd.get_dummies(dfin["vendor_id"].astype(str), prefix="vendor", drop_first=False)
    X = pd.concat([X, vendor_oh], axis=1)

    X = X.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    return X


In [7]:
# Build train/val features and ALIGN columns (critical for get_dummies)
X_train = build_features(train_df)
feature_cols = X_train.columns

X_val = build_features(val_df).reindex(columns=feature_cols, fill_value=0.0)

y_train = train_df[TARGET].to_numpy().astype(np.float64)
y_val   = val_df[TARGET].to_numpy().astype(np.float64)

print("X_train:", X_train.shape, "X_val:", X_val.shape)
print("y_train:", y_train.shape, "y_val:", y_val.shape)


X_train: (333333, 14) X_val: (166667, 14)
y_train: (333333,) y_val: (166667,)


In [8]:
# Scale using TRAIN stats only
mu = X_train.mean()
sigma = X_train.std().replace(0, 1)

X_train_s = (X_train - mu) / sigma
X_val_s   = (X_val   - mu) / sigma

# Save artifacts (optional)
mu.to_csv(ARTIFACTS_DIR / "mu.csv")
sigma.to_csv(ARTIFACTS_DIR / "sigma.csv")
pd.Series(feature_cols, name="feature").to_csv(ARTIFACTS_DIR / "feature_cols.csv", index=False)

print("Saved artifacts to:", ARTIFACTS_DIR.resolve())


Saved artifacts to: /home/jovyan/csusb_spring26_cse5140_team1/artifacts


## Metrics helpers (log space + original scale)
We select/tune models using **log-space** metrics (R²_log, RMSE_log) and report **MAPE** for normalized comparison.

In [9]:
CLIP_MIN = -2.0
CLIP_MAX = 13.0  # tighter cap; prevents absurd durations

def safe_expm1(yhat_log, clip_min=CLIP_MIN, clip_max=CLIP_MAX):
    yhat_log = np.asarray(yhat_log).reshape(-1)
    yhat_log = np.clip(yhat_log, clip_min, clip_max)
    return np.expm1(yhat_log)

def mape(y_true, y_pred, eps=1.0):
    y_true = np.asarray(y_true).reshape(-1).astype(np.float64)
    y_pred = np.asarray(y_pred).reshape(-1).astype(np.float64)
    denom = np.maximum(np.abs(y_true), eps)
    return float(np.mean(np.abs((y_true - y_pred) / denom)) * 100.0)

def report_log(y_true_log, y_pred_log, label=""):
    rmse_log = mean_squared_error(y_true_log, y_pred_log, squared=False)
    r2_log = r2_score(y_true_log, y_pred_log)
    print(label)
    print("  RMSE_log:", rmse_log)
    print("  R2_log  :", r2_log)
    return {"rmse_log": rmse_log, "r2_log": r2_log}

def report_orig(y_true, y_pred, label=""):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae_ = mean_absolute_error(y_true, y_pred)
    r2   = r2_score(y_true, y_pred)
    mape_ = mape(y_true, y_pred)
    print(label)
    print("  RMSE:", rmse)
    print("  MAE :", mae_)
    print("  R^2 :", r2)
    print("  MAPE(%):", mape_)
    return {"rmse": rmse, "mae": mae_, "r2": r2, "mape": mape_}


## Baseline: Ridge regression (trained on log1p target)

In [10]:
y_train_log = np.log1p(y_train)
y_val_log   = np.log1p(y_val)

ridge = Ridge(alpha=1.0, random_state=SEED)
ridge.fit(X_train_s, y_train_log)

val_pred_log_ridge = ridge.predict(X_val_s)
val_pred_log_ridge = np.clip(val_pred_log_ridge, CLIP_MIN, CLIP_MAX)

val_pred_ridge = safe_expm1(val_pred_log_ridge)

baseline_log = report_log(y_val_log, val_pred_log_ridge, "VALIDATION — Baseline (Ridge) log space:")
baseline_orig = report_orig(y_val, val_pred_ridge, "VALIDATION — Baseline (Ridge) original scale:")


VALIDATION — Baseline (Ridge) log space:
  RMSE_log: 0.6261026872904123
  R2_log  : 0.3808989452900038
VALIDATION — Baseline (Ridge) original scale:
  RMSE: 8619.111333223163
  MAE : 514.2058312823963
  R^2 : -0.13513533376099418
  MAPE(%): 71.96278901754312


## Neural Network (PyTorch)
Stabilized training: output clamp + SmoothL1Loss + gradient clipping.

In [11]:
class TabularDataset(Dataset):
    def __init__(self, X, y_log):
        self.X = torch.tensor(np.asarray(X), dtype=torch.float32)
        self.y = torch.tensor(np.asarray(y_log), dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i):
        return self.X[i], self.y[i]

train_ds = TabularDataset(X_train_s.values, y_train_log)
val_ds   = TabularDataset(X_val_s.values,   y_val_log)

train_loader = DataLoader(train_ds, batch_size=2048, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=4096, shuffle=False, num_workers=0)


In [12]:
in_dim = X_train_s.shape[1]

mlp = nn.Sequential(
    nn.Linear(in_dim, 256),
    nn.ReLU(),
    nn.Dropout(0.10),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Dropout(0.10),
    nn.Linear(128, 1),
).to(device)

loss_fn = nn.SmoothL1Loss(beta=0.5)
optimizer = torch.optim.Adam(mlp.parameters(), lr=5e-4, weight_decay=1e-5)

print(mlp)


Sequential(
  (0): Linear(in_features=14, out_features=256, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.1, inplace=False)
  (3): Linear(in_features=256, out_features=128, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.1, inplace=False)
  (6): Linear(in_features=128, out_features=1, bias=True)
)


In [13]:
def evaluate_nn_log(model, Xs_np):
    model.eval()
    with torch.no_grad():
        pred_log = model(torch.tensor(Xs_np, dtype=torch.float32).to(device)).cpu().numpy().reshape(-1)
    pred_log = np.clip(pred_log, CLIP_MIN, CLIP_MAX)
    return pred_log

# Train with early stopping on VAL RMSE_log
best_state = None
best_rmse_log = float("inf")
patience = 3
pat = 0

EPOCHS = 12

for epoch in range(1, EPOCHS+1):
    mlp.train()
    total = 0.0
    n = 0

    for xb, yb in train_loader:
        xb = xb.to(device)
        yb = yb.to(device).view(-1)

        optimizer.zero_grad()
        pred_log = mlp(xb).view(-1)
        pred_log = torch.clamp(pred_log, CLIP_MIN, CLIP_MAX)  # key stabilization
        loss = loss_fn(pred_log, yb)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(mlp.parameters(), max_norm=1.0)
        optimizer.step()

        total += float(loss.item()) * xb.size(0)
        n += xb.size(0)

    train_loss = total / n

    # validation in log space
    val_pred_log_nn = evaluate_nn_log(mlp, X_val_s.values.astype(np.float32))
    val_rmse_log = mean_squared_error(y_val_log, val_pred_log_nn, squared=False)

    print(f"Epoch {epoch:02d} | train_loss={train_loss:.4f} | val_RMSE_log={val_rmse_log:.4f}")

    if val_rmse_log < best_rmse_log - 1e-4:
        best_rmse_log = val_rmse_log
        best_state = {k: v.detach().cpu().clone() for k, v in mlp.state_dict().items()}
        pat = 0
    else:
        pat += 1
        if pat >= patience:
            print("Early stopping.")
            break

# Restore best
if best_state is not None:
    mlp.load_state_dict(best_state)
print("Best VAL RMSE_log:", best_rmse_log)


Epoch 01 | train_loss=1.3420 | val_RMSE_log=0.7132
Epoch 02 | train_loss=0.3420 | val_RMSE_log=0.5440
Epoch 03 | train_loss=0.2881 | val_RMSE_log=0.5099
Epoch 04 | train_loss=0.2689 | val_RMSE_log=0.4968
Epoch 05 | train_loss=0.2579 | val_RMSE_log=0.4897
Epoch 06 | train_loss=0.2501 | val_RMSE_log=0.4862
Epoch 07 | train_loss=0.2441 | val_RMSE_log=0.4816
Epoch 08 | train_loss=0.2407 | val_RMSE_log=0.4803
Epoch 09 | train_loss=0.2380 | val_RMSE_log=0.4801
Epoch 10 | train_loss=0.2344 | val_RMSE_log=0.4756
Epoch 11 | train_loss=0.2304 | val_RMSE_log=0.4749
Epoch 12 | train_loss=0.2269 | val_RMSE_log=0.4721
Best VAL RMSE_log: 0.4720816003370926


In [14]:
# NN validation metrics (log space + original scale)
val_pred_log_nn = evaluate_nn_log(mlp, X_val_s.values.astype(np.float32))
val_pred_nn = safe_expm1(val_pred_log_nn)

print("Max log prediction:", float(np.max(val_pred_log_nn)))
print("Min log prediction:", float(np.min(val_pred_log_nn)))

nn_log = report_log(y_val_log, val_pred_log_nn, "VALIDATION — Neural Net log space:")
nn_orig = report_orig(y_val, val_pred_nn, "VALIDATION — Neural Net original scale:")


Max log prediction: 13.0
Min log prediction: 4.476377010345459
VALIDATION — Neural Net log space:
  RMSE_log: 0.4720816003370926
  R2_log  : 0.6480308851436248
VALIDATION — Neural Net original scale:
  RMSE: 8218.487433101236
  MAE : 376.79086097955735
  R^2 : -0.03206357847685437
  MAPE(%): 37.743075886472475


## Feature importance (Permutation)
We compute permutation importance using **drop in R²_log** on validation to avoid heavy‑tail instability on original scale.

In [15]:
import pandas as pd

def permutation_importance_r2log(predict_log_fn, X_df, y_true_log, n_repeats=3, seed=SEED):
    rng = np.random.default_rng(seed)

    base_pred_log = predict_log_fn(X_df)
    base_r2 = r2_score(y_true_log, base_pred_log)

    importances = []
    X_work = X_df.copy()

    for col in X_df.columns:
        drops = []
        for _ in range(n_repeats):
            saved = X_work[col].to_numpy().copy()
            X_work[col] = rng.permutation(X_work[col].to_numpy())
            pred_log = predict_log_fn(X_work)
            drops.append(base_r2 - r2_score(y_true_log, pred_log))
            X_work[col] = saved
        importances.append((col, float(np.mean(drops))))

    imp = pd.DataFrame(importances, columns=["feature", "r2log_drop"]).sort_values("r2log_drop", ascending=False).reset_index(drop=True)
    return imp, base_r2

# Ridge predict (log space) — keep DataFrame to preserve feature names
def ridge_predict_log_fn(Xdf_raw):
    Xs_df = (Xdf_raw - mu) / sigma
    pred_log = ridge.predict(Xs_df)
    return np.clip(pred_log, CLIP_MIN, CLIP_MAX)

# NN predict (log space)
def nn_predict_log_fn(Xdf_raw):
    Xs = ((Xdf_raw - mu) / sigma).to_numpy().astype(np.float32)
    return evaluate_nn_log(mlp, Xs)

imp_ridge, ridge_r2log = permutation_importance_r2log(ridge_predict_log_fn, X_val, y_val_log, n_repeats=3, seed=SEED)
print("Permutation importance — Baseline (Ridge) on VAL (log space)")
print("Baseline VAL R2_log:", ridge_r2log)
display(imp_ridge.head(20))

imp_nn, nn_r2log = permutation_importance_r2log(nn_predict_log_fn, X_val, y_val_log, n_repeats=3, seed=SEED)
print("Permutation importance — Neural Net on VAL (log space)")
print("NN VAL R2_log:", nn_r2log)
display(imp_nn.head(20))


Permutation importance — Baseline (Ridge) on VAL (log space)
Baseline VAL R2_log: 0.3808989452900038


,feature,r2log_drop
0,haversine_km,0.640220
1,hour_cos,0.024273
2,hour_sin,0.010955
3,dow_cos,0.007680
4,pickup_month,0.003107
5,delta_lon,0.002728
6,dow_sin,0.001783
7,pickup_dow,0.000992
8,passenger_count,0.000311
9,pickup_hour,0.000162


Permutation importance — Neural Net on VAL (log space)
NN VAL R2_log: 0.6480308851436248


,feature,r2log_drop
0,vendor_1,1.936115
1,vendor_2,1.927551
2,haversine_km,1.652740
3,hour_cos,0.624782
4,pickup_hour,0.265949
5,dow_cos,0.220403
6,pickup_dow,0.165449
7,hour_sin,0.162388
8,dow_sin,0.158319
9,delta_lat,0.140979


## Final evaluation on 50% holdout (touch once)
Report both log-space and original-scale metrics. For rubric-style normalized comparison, compare **R²_log + MAPE**.

In [16]:
# Build holdout features ONLY here
X_holdout = build_features(holdout_df).reindex(columns=feature_cols, fill_value=0.0)
y_holdout = holdout_df[TARGET].to_numpy().astype(np.float64)
y_holdout_log = np.log1p(y_holdout)

X_holdout_s = (X_holdout - mu) / sigma

# Baseline holdout
hold_pred_log_ridge = ridge.predict(X_holdout_s)
hold_pred_log_ridge = np.clip(hold_pred_log_ridge, CLIP_MIN, CLIP_MAX)
hold_pred_ridge = safe_expm1(hold_pred_log_ridge)

print("HOLDOUT — Baseline (Ridge)")
hold_base_log = report_log(y_holdout_log, hold_pred_log_ridge, "  log space:")
hold_base_orig = report_orig(y_holdout, hold_pred_ridge, "  original scale:")

# NN holdout
hold_pred_log_nn = evaluate_nn_log(mlp, X_holdout_s.to_numpy().astype(np.float32))
hold_pred_nn = safe_expm1(hold_pred_log_nn)

print("\nHOLDOUT — Neural Net")
hold_nn_log = report_log(y_holdout_log, hold_pred_log_nn, "  log space:")
hold_nn_orig = report_orig(y_holdout, hold_pred_nn, "  original scale:")

print("\nNormalized comparison (rubric-style):")
print("Baseline: R2_log =", hold_base_log["r2_log"], "| MAPE% =", hold_base_orig["mape"])
print("NN      : R2_log =", hold_nn_log["r2_log"],   "| MAPE% =", hold_nn_orig["mape"])


HOLDOUT — Baseline (Ridge)
  log space:
  RMSE_log: 0.6254690523724664
  R2_log  : 0.38097988445067077
  original scale:
  RMSE: 7178.442827412152
  MAE : 492.363648691323
  R^2 : -0.22099133819017092
  MAPE(%): 73.75791670847394

HOLDOUT — Neural Net
  log space:
  RMSE_log: 0.46814532519091284
  R2_log  : 0.6532196410745756
  original scale:
  RMSE: 6720.777562595433
  MAE : 354.7781077965698
  R^2 : -0.07026453611091776
  MAPE(%): 38.362744179328644

Normalized comparison (rubric-style):
Baseline: R2_log = 0.38097988445067077 | MAPE% = 73.75791670847394
NN      : R2_log = 0.6532196410745756 | MAPE% = 38.362744179328644
